In [1]:
import bs4
import time
import platform
from datetime import datetime

In [2]:
import get_information

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.keys import Keys

In [4]:
# INSTANCE BROWSER
def openBrowser(business):
    '''
        Verifica qual o sistema operacional para que use o driver e diretorio de usuarios certo
    '''
    
    if platform.system() == 'Linux':
        driver_path = './webdriver/linux/chromedriver'
        profile_path = '/home/nomadbitcoin/whatsapp_profiles/' + business 
    elif platform.system() == 'Windows':
        path = './webdriver/win/chromedriver.exe'
        profile_path = 'Need Implement'
    
    
    global driver
    try:
        options = webdriver.ChromeOptions()
        # open with saved informations in cache
        options.add_argument("profile")
        options.add_argument("user-data-dir=" + profile_path)
        driver = webdriver.Chrome(options=options, executable_path=driver_path)
#         driver.implicitly_wait(5) #tempo implicito de espera 1 segundo antes de cada acao
        main_url = 'https://web.whatsapp.com/'
        driver.get(main_url)
        if wait_connection(driver):
            return True, 'opened in: {}'.format(main_url)
    except Exception as error:
        print(type(error), error)
        return False
    
# PARA LIBERAR SOMENTE APOS TER ABERTO O WHATSAPP
def wait_connection(driver):
    waiting = True
    while waiting:
        try:
            if driver.find_element_by_class_name('Qk8nZ').text == 'Keep your phone connected':
                waiting = False
                return True
                break
        except:
            time.sleep(10)
            pass

In [5]:
openBrowser('nomadbitcoin')

(True, 'opened in: https://web.whatsapp.com/')

In [6]:
gdata = get_information.get_Data(driver)

In [11]:
chats = gdata.getChats()

In [39]:
def getPageSource():
    global htmlPage
    htmlPage = gdata.getPageSource(True)

In [41]:
def slc_get_messages():
    '''
        Pega o historico de mensagens que esta carregada no documento
    '''
    getPageSource()
    try:
        messages = htmlPage.find_all('div',{'class':'FTBzM'})
        assert messages != [], 'Not Chat Selected'
        return messages
    except Exception as error:
        print(error)
        pass

In [124]:
def msg_type_content(message):
    '''
        Recebe uma mensagem e verifica qual o formato do conteudo: 
            *texto, audio, imagem, video, link, documento, localizacao ou contato
    '''
    getPageSource()
    
    if message.find('span',{'data-icon':'audio-play'}) != None:
        msg_type = 'audio'
    elif message.find('div',{'class':'_1o0MN'}) != None: #testar para outros tipos de documentos
        msg_type = 'document'
    elif message.find('img') != None:
        msg_type = 'image'
    elif message.find('a') != None:
        msg_type = 'link'
    elif message.find('span',{'data-icon':'msg-video-light'}) != None:
        msg_type = 'video'
    else:
        msg_type = 'text'
    return msg_type

In [132]:
messages = slc_get_messages()
time.sleep(1)
msg_type_content(messages[-1])

'audio'

<span class="" data-icon="msg-video-light"><svg height="14" viewbox="0 0 16 14" width="16" xmlns="http://www.w3.org/2000/svg"><path d="M14.987 2.668l-3.48 3.091v-2.27c0-.657-.532-1.189-1.189-1.189H1.689C1.032 2.3.5 2.832.5 3.489v7.138c0 .657.532 1.189 1.189 1.189h8.629c.657 0 1.189-.532 1.189-1.189V8.328l3.48 3.09v-8.75z" fill="#FFF"></path></svg></span>

In [43]:
def slc_get_last_msg():
    '''
        Pega a ultima mensagem do historico, o tipo do conteudo, horario e status
    '''
    getPageSource()
    messages = slc_get_messages()
    msg_time = gdata.getTime(messages[-1], 'message')
    return messages[-1]